In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import art
from art.local import LocalBackend

model = art.TrainableModel(
    name="test", project="tests", base_model="Qwen/Qwen2.5-7B-Instruct"
)
await model.register(LocalBackend())

wandb: Currently logged in as: bradhilton to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


INFO 08-21 18:37:49 [__init__.py:235] Automatically detected platform cuda.


/home/ubuntu/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-21 18:37:59 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.6: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA H100 PCIe. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.47%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.19 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill toke

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00, 27.85it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  2.52it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.39it/s]



INFO 08-21 18:38:44 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 08-21 18:38:45 [model_runner.py:1115] Model loading took 6.7355 GiB and 17.780424 seconds
INFO 08-21 18:38:54 [worker.py:295] Memory profiling takes 8.58 seconds
INFO 08-21 18:38:54 [worker.py:295] the current vLLM instance can use total_gpu_memory (79.19GiB) x gpu_memory_utilization (0.78) = 62.14GiB
INFO 08-21 18:38:54 [worker.py:295] model weights take 6.74GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 4.72GiB; the rest of the memory reserved for KV Cache is 50.54GiB.
INFO 08-21 18:38:54 [executor_base.py:113] # cuda blocks: 59142, # CPU blocks: 7021
INFO 08-21 18:38:54 [executor_base.py:118] Maximum concurrency for 32768 tokens per request: 28.88x
INFO 08-21 18:38:58 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-21 18:38:58 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To 

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:29<00:00,  1.66it/s]


INFO 08-21 18:39:27 [model_runner.py:1537] Graph capturing finished in 30 secs, took 1.35 GiB
INFO 08-21 18:39:27 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 30 secs.
INFO 08-21 18:39:28 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 43.07 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm', 'q_norm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm', 'q_norm']


Unsloth 2025.8.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
client = model.openai_client()

chat_completion = await client.chat.completions.create(
    model="test",
    messages=[{"role": "user", "content": "Hi!"}],
)

In [7]:
chat_completion.model_dump(mode="json")

{'id': 'chatcmpl-293ce9f37dba40e5be39448acaf6fb49',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': {'content': [{'token': 'token_id:9707',
      'bytes': [72, 101, 108, 108, 111],
      'logprob': -0.0017243054462596774,
      'top_logprobs': []},
     {'token': 'token_id:0',
      'bytes': [33],
      'logprob': -0.007611795328557491,
      'top_logprobs': []},
     {'token': 'token_id:2585',
      'bytes': [32, 72, 111, 119],
      'logprob': -0.03061593696475029,
      'top_logprobs': []},
     {'token': 'token_id:646',
      'bytes': [32, 99, 97, 110],
      'logprob': -1.1920858014491387e-05,
      'top_logprobs': []},
     {'token': 'token_id:358',
      'bytes': [32, 73],
      'logprob': -2.3841855067985307e-07,
      'top_logprobs': []},
     {'token': 'token_id:7789',
      'bytes': [32, 97, 115, 115, 105, 115, 116],
      'logprob': -0.020548323169350624,
      'top_logprobs': []},
     {'token': 'token_id:498',
      'bytes': [32, 121, 111, 117],
     